<a href="https://colab.research.google.com/github/sbb2002/Portfolio/blob/main/.study/Tensorflow/ML_ch8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ch 8. Tensor manipulation

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


# The function dictionary (tf v1.0)

## Array

In [ ]:
import numpy as np

# 1D-array
t1 = np.array([0,1,2,3,4,5,6,7,8,9])

In [ ]:
t1.ndim   # dimension (rank)

1

In [ ]:
t1.shape  # number of elems in each dimension; Output=(..., 2d, 1d,)

(10,)

In [ ]:
# 2D-array
t2 = np.array([[0,1,2], [3,4,5], [6,7,8], [9,10,11]])

In [ ]:
t2.ndim

2

In [ ]:
t2.shape

(4, 3)

* rank
 > dimension, 수식 앞의 " [ " 갯수

* shape
 > dimension 안 elem의 갯수, 높은 차원 순으로 기술 \
 ex. (rank 3, rank 2, rank 1)

* axis
 > rank-depth. \
 겉의 " [ "는 axis=0, \
 그 다음 " [[ "는 axis=1, \
 ... , \
 마지막 " [...[ "는 axis=n or -1

In [ ]:
t = tf.constant([
                 [
                  [
                   [1,2,3,4],
                   [5,6,7,8],
                   [9,10,11,12]
                  ],
                  [
                   [13,14,15,16],
                   [17,18,19,20],
                   [21,22,23,24]
                  ]
                ]
              ])

In [ ]:
tf.shape(t).eval(session=tf.Session())

array([1, 2, 3, 4], dtype=int32)

In [ ]:
# 위의 출력값을 설명
e1, e2, e3, e4 = tf.shape(t).eval(session=tf.Session())
print("[\t*\t]\t안에 {} 개의 [[[\t]]]\t가 있고, ".format(e1))
print("[[\t*\t]]\t안에 {} 개의 [[\t]]\t가 있고, ".format(e2))
print("[[[\t*\t]]]\t안에 {} 개의 [\t]\t가 있고, ".format(e3))
print("[[[\t*\t]]]]\t안에 {} 개의 elem\t가 있다. ".format(e4))


[	*	]	안에 1 개의 [[[	]]]	가 있고, 
[[	*	]]	안에 2 개의 [[	]]	가 있고, 
[[[	*	]]]	안에 3 개의 [	]	가 있고, 
[[[	*	]]]]	안에 4 개의 elem	가 있다. 


## tf.matmul VS multiply

In [ ]:
m1 = tf.constant([[1,2], [3,4]])
m2 = tf.constant([[1], [2]])

print("m1의 shape은 ", m1.shape)
print("m2의 shape은 ", m2.shape)
print("\n행렬곱(내적) m1 . m2 을 하려면 'm1의 shape-2nd' 와 'm2의 shape-1st'가 같아야합니다.")
tf.matmul(m1, m2).eval(session=tf.Session())

m1의 shape은  (2, 2)
m2의 shape은  (2, 1)

행렬곱(내적) m1 x m2 을 하려면 'm1의 shape-2nd' 와 'm2의 shape-1st'가 같아야합니다.


array([[ 5],
       [11]], dtype=int32)

In [ ]:
print("단순곱은 hadamard product를 합니다. 내적과 비슷하므로 유의하세요!")
(m1 * m2).eval(session=tf.Session())

단순곱은 hadamard product를 합니다. 내적과 비슷하므로 유의하세요!


array([[1, 2],
       [6, 8]], dtype=int32)

## Broadcasting

matrix의 rank나 shape이 맞지 않아도 자동완성해서 계산에 하자가 없게 해주는 기능이다. \
단, 원리를 잘못 파악하면 상당한 독이 될 수 있다. \
왠만하면 쓰지말자...

In [ ]:
m1 = tf.constant([[1,2]])
m2 = tf.constant(3)
(m1 + m2).eval(session=tf.Session())

array([[4, 5]], dtype=int32)

In [ ]:
m1 = tf.constant([[1,2]])
m2 = tf.constant([3,4])
(m1 + m2).eval(session=tf.Session())

array([[4, 6]], dtype=int32)

In [ ]:
m1 = tf.constant([[1,2]])
m2 = tf.constant([[3], [4]])
(m1 + m2).eval(session=tf.Session())

array([[4, 5],
       [5, 6]], dtype=int32)

## Reduce mean & sum

In [ ]:
print("평균 1.5가 나올 것 같지만, 출력type이 int32이기 때문에 소수점을 버린 1이 나옵니다. 만약 주어진 데이터가 1. 또는 2. 과 같은 float32이라면 거기에 맞춥니다.")

x = tf.reduce_mean([1,2], axis=0).eval(session=tf.Session())
y = type(tf.reduce_mean([1,2], axis=0).eval(session=tf.Session()))
print("계산: ", x, "\tType: ", y)

x = tf.reduce_mean([1.,2.], axis=0).eval(session=tf.Session())
y = type(tf.reduce_mean([1.,2.], axis=0).eval(session=tf.Session()))
print("계산: ", x, "\tType: ", y)

평균 1.5가 나올 것 같지만, 출력type이 int32이기 때문에 소수점을 버린 1이 나옵니다. 만약 주어진 데이터가 1. 또는 2. 과 같은 float32이라면 거기에 맞춥니다.
계산:  1 	Type:  <class 'numpy.int32'>
계산:  1.5 	Type:  <class 'numpy.float32'>


새로 다음과 같은 데이터를 주자.

```
[
  [1. ,2. ], 
  [3. ,4. ]
]
```

reduce_mean or sum은 연산을 수행할 때 이러한 특징이 있다.
> 만약 대상으로 하는 axis의 구성원이 ...\
elem일 때는 단순히 모두 합하든 평균을 내든 하지만, \
list일 경우 또 그 안의 elem들을 같은 idx끼리 연산시킨다.

* axis = 0
 * 세로 방향
 * 첫 번째 [ * ] 안의 elem(_list)을 대상으로 연산한다. \

* axis = 1
 * 가로 방향
 * 두 번째 [ * ] 안의 elem(_list)을 대상으로 연산한다.


In [ ]:
tf.reduce_mean([[1. ,2. ], [3. ,4. ]], axis=0).eval(session=tf.Session())

array([2., 3.], dtype=float32)

In [ ]:
tf.reduce_mean([[1. ,2. ], [3. ,4. ]], axis=1).eval(session=tf.Session())

array([1.5, 3.5], dtype=float32)

In [ ]:
tf.reduce_mean([[1. ,2. ], [3. ,4. ]], axis=None).eval(session=tf.Session())

2.5

axis가 None일 때(default) 전부 다 elem취급하여 연산한다.

reduce sum도 마찬가지.

In [ ]:
tf.reduce_sum([[1. ,2. ], [3. ,4. ]], axis=0).eval(session=tf.Session())

array([4., 6.], dtype=float32)

In [ ]:
tf.reduce_sum([[1. ,2. ], [3. ,4. ]], axis=-1).eval(session=tf.Session())

array([3., 7.], dtype=float32)

In [ ]:
tf.reduce_sum([[1. ,2. ], [3. ,4. ]]).eval(session=tf.Session())

10.0

In [ ]:
tf.reduce_sum([[1. ,2. ], [3. ,4. ]], axis=None).eval(session=tf.Session())

10.0

## argmax: one-hot encoding!

In [ ]:
x = [[0,1,2],
     [2,1,0]]
tf.argmax(x, axis=0).eval(session=tf.Session())

array([1, 0, 0])

In [ ]:
tf.argmax(x, axis=1).eval(session=tf.Session())

array([2, 0])

## Reshape + squeeze, expand

In [ ]:
t = np.array([[[0,1,2],
               [3,4,5]],
              [[6,7,8],
               [9,10,11]]])
t.shape

(2, 2, 3)

In [ ]:
tf.reshape(t, shape=[-1, 3]).eval(session=tf.Session())   # shape=[-1, 3] means -1: auto, 3: want number of elems

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [ ]:
tf.reshape(t, shape=[-1,1, 3]).eval(session=tf.Session()) 

array([[[ 0,  1,  2]],

       [[ 3,  4,  5]],

       [[ 6,  7,  8]],

       [[ 9, 10, 11]]])

squeeze는 matrix를 쫙 펴준다. (rank -= 1)

In [ ]:
tf.squeeze([[0], [1], [2]]).eval(session=tf.Session()) 

array([0, 1, 2], dtype=int32)

expand_dims는 거꾸로 matrix를 한 차원 늘려준다. (rank += 1)

In [ ]:
tf.expand_dims([0, 1, 2], 1).eval(session=tf.Session()) 

array([[0],
       [1],
       [2]], dtype=int32)

## (Real) One-hot: 자리번호

In [ ]:
tf.one_hot([[0], [1], [2], [0]], depth=3).eval(session=tf.Session())      # depth=(자리 갯수)

array([[[1., 0., 0.]],

       [[0., 1., 0.]],

       [[0., 0., 1.]],

       [[1., 0., 0.]]], dtype=float32)

전에 본대로 rank += 1을 한다.

## Casting

In [ ]:
tf.cast([1.8, 2.2, 3.7, 4.9], tf.int32).eval(session=tf.Session()) 

array([1, 2, 3, 4], dtype=int32)

In [ ]:
tf.cast([True, False, 1==1, 0==1], tf.int32).eval(session=tf.Session()) 

array([1, 0, 1, 0], dtype=int32)

유리수에서 정수값만 떼와주거나, 참/거짓을 1/0으로 나타내준다.

## Stack

In [ ]:
x = [1, 4]
y = [2, 5]
z = [3, 6]
w = [4, 7]

# Pack along first dim
tf.stack([x,y,z,w]).eval(session=tf.Session())

array([[1, 4],
       [2, 5],
       [3, 6],
       [4, 7]], dtype=int32)

In [ ]:
tf.stack([x,y,z,w]).eval(session=tf.Session()).shape

(4, 2)

In [ ]:
# Pack along second dim
tf.stack([x,y,z,w], axis=1).eval(session=tf.Session()) 

array([[1, 2, 3, 4],
       [4, 5, 6, 7]], dtype=int32)

In [ ]:
tf.stack([x,y,z,w], axis=1).eval(session=tf.Session()).shape

(2, 4)

stack 안에 [ * ]를 주목하자. \
거기 들어간 변수 갯수만큼 해당 axis의 elem 갯수가 되게 해준다. shape를 유심히 볼 것.

## One & Zeros like

In [ ]:
x = [[0,1,2],
     [2,1,0]]
tf.ones_like(x).eval(session=tf.Session()) 

array([[1, 1, 1],
       [1, 1, 1]], dtype=int32)

In [ ]:
tf.zeros_like(x).eval(session=tf.Session()) 

array([[0, 0, 0],
       [0, 0, 0]], dtype=int32)

shape만 따와서 그 속을 0이나 1로 채움.

## Zip

In [ ]:
for x, y in zip([1,2,3], [4,5,6]):
  print(x, y)

1 4
2 5
3 6


In [ ]:
for x, y, z in zip([1,2,3], [4,5,6], [7,8,9]):
  print(x, y, z)

1 4 7
2 5 8
3 6 9
